<center><h2>Академия Аналитиков Авито 2024-2025<br><br>Домашнее задание по Audio Processing</h2></center>

In [ ]:
# !pip3.11 install audiomentations

In [1]:
!pip install jiwer evaluate 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.9 MB/s eta 0:00:00:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-c

In [6]:
torch.cuda.is_available()

True

In [5]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

if "src" in os.getcwd():
    os.chdir("../")
import pandas as pd
import torch
import numpy as np
import torchaudio
import jiwer
from IPython.display import display, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from pathlib import Path
from tqdm import tqdm

In [7]:
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

In [8]:
# from torch_audiomentations import (
#     Compose, 
#     Lambda,
#     AddGaussianNoise,
#     AddBackgroundNoise, 
#     ApplyImpulseResponse,
#     Gain, 
#     AddColorNoise, 
#     BitCrush, 
#     OneOf
# )

from datasets import DatasetDict, Dataset
from datasets import Audio as DatasetsAudio

In [9]:
from dataclasses import dataclass
from typing import Dict, List, Union, Any
import evaluate

#### 🛠️ Утилиты для домашки

↓ Скачивание набора данных RIR-ов и шумов. Можно использовать другой набор данных.

In [10]:
# # Качаем и извлекаем базу шумов и импульсных характеристик
# !curl -L -o "./data/datasets/rirs_and_noises.zip" "https://www.openslr.org/resources/28/rirs_noises.zip"
# !unzip "./data/datasets/rirs_and_noises.zip" -d "./data/datasets"

↓ Утилиты работы с обученной моделью

In [11]:
def postprocess_text(text: str) -> str:
    """Постобработка строк. 
    Приводим в нижний регистр, удаляем краевые пробельные 
    и неалфавитные символы (цифры, пунктуацию).
    """

    text = text.lower().strip()
    text = "".join(s for s in text if str(s).isalpha() or s == " ")

    return text


def infer_whisper(
    model: WhisperForConditionalGeneration,
    processor: WhisperProcessor,
    wav: torch.Tensor, 
    sample_rate: int, 
    lang: str = "ru",
    device: str = "cuda:0"
) -> str:
    """Обработка Whisper-ом входного аудио."""

    # Обрабатываем аудио
    inputs = processor(
        wav.squeeze().numpy(),
        sampling_rate=sample_rate,
        return_tensors="pt",
    )

    # Генерируем транскрипцию
    outputs = model.generate(
        **inputs.to(device),
        language=f"<|{lang}|>"
    )

    hypothesis = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    return hypothesis

↓ Подсчёт Keyword Error Rate

In [12]:
def compute_ker(reference: str, hypothesis: str, keyword: str) -> float:
    reference = reference.lower()
    hypothesis = hypothesis.lower()
    keyword = keyword.lower()

    ref_keywords = [word for word in reference.split() if word == keyword]
    hyp_keywords = [word for word in hypothesis.split() if word == keyword]

    ref_keywords = " ".join(ref_keywords)
    hyp_keywords = " ".join(hyp_keywords)

    wer_transforms = jiwer.transforms.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.ReduceToListOfListOfWords()
    ])

    if len(ref_keywords) == 0:
        raise ValueError("There is no keywords in reference.")
    
    ker = jiwer.wer(
        ref_keywords, 
        hyp_keywords, 
        reference_transform=wer_transforms, 
        hypothesis_transform=wer_transforms
    )

    return ker

### **Формулировка задачи**

#### 💢 Проблема

Слово "Авито" распознаётся плохо оригинальным Whisper-ом. Либо он пишет его на латинице, либо не в той форме, либо путает с другими словами.

Пример:

In [13]:
torch.cuda.is_available()

True

In [14]:
device = "cuda:0"

# Инициализируем модель
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.to(device)
processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Читаем аудио
wav, sr = torchaudio.load("/kaggle/input/audiodata/audio_proccesing/audio_homework/data/audio/hello_avito.wav",)
display(Audio(data=wav, rate=sr))

# Рэсемплим в 16кГц (Whisper по-другому не умеет)
wav = torchaudio.transforms.Resample(sr, 16000)(wav)

# Запускаем модель на аудио
hypothesis = infer_whisper(
    model=model,
    processor=processor,
    wav=wav, 
    sample_rate=16000, 
    lang="ru",
    device=device
)
postprocess_text(hypothesis)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

You have passed language=<|ru|>, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=<|ru|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'вас приветствует компания avita'

#### 😼 Что нужно сделать?

In [15]:
torchaudio.list_audio_backends()

['ffmpeg', 'soundfile']

<span style="color: #cc6104;">От вас требуется заставить модель `openai/whisper-small` говорить слово "авито" в надлежащей форме (именительный падеж, единственное число).</span>

Можно пользоваться любыми способами, кроме очевидных:
1. Делать постпроцессинг распознанной строки, применяя регулярные выражения (или еще что-нибудь).
2. Накладывать ограничения на логиты во время генерации текста моделью (constrained decoding).

Любое другое соображение предлагаю обсуждать в личных сообщениях со мной, чтобы я давал добро или накладывал на него вето.

В общем случае ваша модель должна сама, "от природы", генерировать правильную форму слова "авито" там, где оно было произнесено.

#### 🧑‍🏫️ Описание прикладной области

Система будет работать с 8 кГц моноканальным аудио русской речи, записанной с микрофона телефона во время диалога. На каждой записи находится только один диктор -- другой собеседник ему не мешает. Диктор находится близко к микрофону, речь разборчива (как в примере выше). Часто диктор идёт по улице, едет в транспорте или сидит где-нибудь в столовой. Для экономии места записи были сконвертированы в mp3-формат.

#### 🤖 Формат решения

Ожидаю архив с двумя вещами:
1. Чекпоинт модели: её веса + конфиги processor-а. В общем случае, это папка, которая создается в процессе или результате обучения модели.
2. Python-скрипт с функцией `transcribe`, принимающей на вход одну пару "аудио, частота сэмплирования" и возвращающей одну строку -- транскрипцию.

#### 🥇 По какой метрике я буду вести оценку качества?

[Keyword Error Rate (KER)](https://www.isca-archive.org/interspeech_2008/park08b_interspeech.pdf):

$$KER = \frac{N_{miss} + N_{detect}}{L_{ref}}$$

где 
- $N_{miss}$ -- количество ошибочно упущенных ключевых слов; 
- $N_{detect}$ -- количество ошибочно распознанных ключевых слов; 
- $L_{ref}$ -- количество ключевых слов в референсе.

Имплементация приведена в разделе "Утилиты".

In [16]:
import pandas as pd
import numpy as np
import re

# data = pd.read_csv('/kaggle/input/audiodata/audio_proccesing/avito_replics.csv')
# data = data.rename(columns={"реплика": "text", "содержит_авито": "is_avito"})
# audio_dir = '/kaggle/input/audiodata/audio_proccesing/avito_audio'
# nums = np.arange(7, 1007)

# is_avito_marker = data.is_avito.values.astype(int)
# file_names = []

# for i in range(len(nums)):
#     file_names.append(f"{audio_dir}/avito_{nums[i]}_{is_avito_marker[i]}.wav")

# data['file_name'] = file_names
# data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())

# def extract_dataset_dict_from_directory(data, aug=False):
#     output_audio_dir = "/kaggle/input/audiodata/audio_proccesing/avito_audio_augumented"
#     output = {"audio": [], "text": []}

#     # Проходимся по всем строкам в файле разметки и сохраняем 1 - путь до аудио, 2 - текст из аудио
#     i = 7
#     num_i = 0
#     for row in data.iterrows():
#         if row[1][1]:
#             if aug:
#                 output["audio"].append(f"{output_audio_dir}/avito_aug_{i}_{int(row[1][1])}.wav")
#             else:
#                 output["audio"].append(row[1][2])
#             output["text"].append(row[1][0])
#             num_i += 1
#         if num_i >= 100:
#             break
#         i += 1
    
#     return output

In [17]:
data = pd.read_csv('/kaggle/input/audiodata/audio_proccesing/avito_replics.csv')
data = data.rename(columns={"реплика": "text", "содержит_авито": "is_avito"})

In [18]:
# data = data[data.is_avito==True][:100]

In [19]:
data

,text,is_avito
0,Телефон почти разрядился.,False
1,Поставил ноутбук на авито — уже звонят.,True
2,Ты видел новые объявления на авито сегодня?,True
3,"Скачай приложение — авито, и посмотри сам.",True
4,Не забудь оплатить интернет до вечера.,False
...,...,...
995,Этот стул я на авито нашёл случайно.,True
996,Обожаю находить винтажные вещи на авито.,True
997,"Авито — отличное место, чтобы избавиться от хл...",True
998,Мне нужно срочно в магазин.,False


In [20]:
audio_dir = '/kaggle/input/audiodata/audio_proccesing/avito_audio'

In [21]:
nums = np.arange(7, 1007)
is_avito_marker = data.is_avito.values.astype(int)
file_names = []
for i in range(len(nums)):
    file_names.append(f"{audio_dir}/avito_{nums[i]}_{is_avito_marker[i]}.wav")

In [22]:
data['file_name'] = file_names
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x).lower())

In [23]:
data

,text,is_avito,file_name
0,телефон почти разрядился,False,/kaggle/input/audiodata/audio_proccesing/avito...
1,поставил ноутбук на авито уже звонят,True,/kaggle/input/audiodata/audio_proccesing/avito...
2,ты видел новые объявления на авито сегодня,True,/kaggle/input/audiodata/audio_proccesing/avito...
3,скачай приложение авито и посмотри сам,True,/kaggle/input/audiodata/audio_proccesing/avito...
4,не забудь оплатить интернет до вечера,False,/kaggle/input/audiodata/audio_proccesing/avito...
...,...,...,...
995,этот стул я на авито нашёл случайно,True,/kaggle/input/audiodata/audio_proccesing/avito...
996,обожаю находить винтажные вещи на авито,True,/kaggle/input/audiodata/audio_proccesing/avito...
997,авито отличное место чтобы избавиться от хлама,True,/kaggle/input/audiodata/audio_proccesing/avito...
998,мне нужно срочно в магазин,False,/kaggle/input/audiodata/audio_proccesing/avito...


In [24]:
# # Создаем объект-аугментатор
# augs = Compose([
#     OneOf([AddGaussianNoise(max_amplitude=0.02), AddColorNoise()], p=0.5),
#     Lambda(lambda x, _: x[0], p=1.0),
#     AddBackgroundNoise(
#         sounds_path="/kaggle/input/audiodata/audio_proccesing/RIRS_NOISES/pointsource_noises", 
#         min_snr_db=2.0, 
#         max_snr_db=10.0,
#         p=1.0
#     ),
#     ApplyImpulseResponse(
#         ir_path=list((Path("/kaggle/input/audiodata/audio_proccesing/RIRS_NOISES/simulated_rirs/largeroom")).glob("**/*.wav")),
#         p=1.0
#     ),
#     Gain(p=1.0),
#     BitCrush(p=0.1)
# ])

In [25]:
# # Утилиты
# def pure_filename(file_path: Path) -> str:
#     return ".".join(file_path.name.split(".")[:-1])

In [26]:
# # Подготавливаем выходные директории
# output_audio_dir = "/Users/kirill/edu/aaa_ds/ml_advanced/audio_proccesing/avito_audio_augumented"
# i = 7
# for row in tqdm(data.iterrows()):
#     # Читаем аудио и накладываем аугментации
#     wav, sr = torchaudio.load(row[1][2], normalize=True)  # Нормализуем амплитуду сигнала
#     wav_augmented = augs(wav.numpy(), sr)
#     wav_augmented = torch.from_numpy(wav_augmented).unsqueeze(0)

#     # Сохраняем новое аудио
#     torchaudio.save(f"{output_audio_dir}/avito_aug_{i}_{int(row[1][1])}.wav", wav_augmented, sr)
#     i += 1

In [27]:
def extract_dataset_dict_from_directory(data, aug=False):
    output_audio_dir = "/kaggle/input/audiodata/audio_proccesing/avito_audio_augumented"
    output = {"audio": [], "text": []}

    # Проходимся по всем строкам в файле разметки и сохраняем 1 - путь до аудио, 2 - текст из аудио
    i = 7
    num_i = 0
    for row in data.iterrows():
        if row[1][1]:
            if aug:
                output["audio"].append(f"{output_audio_dir}/avito_aug_{i}_{int(row[1][1])}.wav")
            else:
                output["audio"].append(row[1][2])
            output["text"].append(row[1][0])
            num_i += 1
        if num_i >= 100:
            break
        i += 1
    
    return output

In [28]:
# Создаём объект датасета
asr_dataset = DatasetDict({
    "train": Dataset.from_dict(extract_dataset_dict_from_directory(data, aug=True)),
    "test": Dataset.from_dict(extract_dataset_dict_from_directory(data)),
}).cast_column("audio", DatasetsAudio())

/tmp/ipykernel_31/832423191.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[1][1]:
/tmp/ipykernel_31/832423191.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output["audio"].append(f"{output_audio_dir}/avito_aug_{i}_{int(row[1][1])}.wav")
/tmp/ipykernel_31/832423191.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  output["text"].append(row[1][0])
/tmp/ipykernel_31/832423191.py:13: FutureWarning: Series.__getitem

In [29]:
asr_dataset['train']['text']

['поставил ноутбук на авито  уже звонят',
 'ты видел новые объявления на авито сегодня',
 'скачай приложение  авито и посмотри сам',
 'я на авито заказал новые колёса',
 'объявление на авито быстро набрало просмотры',
 'обожаю находить винтажные вещи на авито',
 'объявление на авито быстро набрало просмотры',
 'купил стол на авито  почти новый',
 'я на авито заказал новые колёса',
 'я на авито нашёл машину дешевле рынка',
 'я продал диван на авито буквально за день',
 'на авито есть всё даже кухонные раковины',
 'скачай приложение  авито и посмотри сам',
 'мне на авито попался редкий фотоаппарат',
 'объявление на авито быстро набрало просмотры',
 'объявление на авито быстро набрало просмотры',
 'помоги мне выбрать фотообъявление для авито',
 'переписывался с продавцом на авито весь вечер',
 'обожаю находить винтажные вещи на авито',
 'я продал диван на авито буквально за день',
 'переписывался с продавцом на авито весь вечер',
 'нашёл классную гитару на авито',
 'авито  отличное место 

In [30]:
# Создаем компоненты модели с целью их применения при подготовке датасета и обучении модели
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ru", task="transcribe")

In [31]:
# Подгружаем саму модель и настраиваем ее
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [32]:
for param in model.model.encoder.parameters():
    param.requires_grad = False

for param in model.model.decoder.embed_tokens.parameters():
    param.requires_grad = False
for param in model.model.decoder.embed_positions.parameters():
    param.requires_grad = False
N = 10  # например, 6 из 12
for i, block in enumerate(model.model.decoder.layers):
    if i < N:
        for param in block.parameters():
            param.requires_grad = False

In [33]:
for param in model.base_model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [34]:
# Описываем логику подготовки датасета
def prepare_dataset(batch):
    audio = batch["audio"]
    
    # Извлекаем признаки. У Whisper -- это 64 мел-коэффициента
    batch["input_features"] = feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Токенизируем
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch


# Применяем описанную функцию подготовки к датасету.
# Удаляем колонки audio и text -- они нам больше не понадобятся. 
# Вместо них держим input_features (мел-спектры) и labels (токены текстовок)
vectorized_asr_dataset = asr_dataset.map(
    prepare_dataset,
    remove_columns=asr_dataset["train"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [35]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    feature_extractor: Any
    tokenizer: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch
    

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    decoder_start_token_id=model.config.decoder_start_token_id
)

In [38]:
# Описываем параметры обучения
training_args = Seq2SeqTrainingArguments(
    # Здесь будут лежать логи и чекпоинты модели, появляющиеся по мере обучения
    output_dir="./finetuning-logs",
    report_to = [],
    # Аргументы, связанные с процессом оптимизации
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    warmup_steps=25,
    max_steps=100,
    gradient_checkpointing=True,
    optim="adamw_torch",
    fp16=True,
    
    # Аргументы, связанные с генерацией транскрипции
    predict_with_generate=True,
    generation_max_length=225,

    # Аргументы, связанные с эвал-шагом
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    save_steps=25,
    # eval_steps=5,

    # Аргументы, связанные с логгированием
    logging_steps=25,
    # report_to=["tensorboard"],
    
    # Аргументы, связанные с оценкой метриками и отбором моделей
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
)

In [39]:
# Определяем функционал подсчета метрик
metric = evaluate.load("wer")


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # Replace -100 with pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    
    # Decode predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [40]:
# Подготавливаем объект-trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    # device="cuda:0",
    train_dataset=vectorized_asr_dataset["train"],
    eval_dataset=vectorized_asr_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=feature_extractor,
)

/tmp/ipykernel_31/3367077751.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [41]:
# model.base_model.parameters

In [42]:
# Запускаем обучение
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
25,1.304800,1.269777,55.864198
50,1.319200,1.269670,56.018519
75,1.306800,1.269578,56.018519
100,1.264400,1.269558,56.018519


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarni

TrainOutput(global_step=100, training_loss=1.2988064765930176, metrics={'train_runtime': 630.3058, 'train_samples_per_second': 5.077, 'train_steps_per_second': 0.159, 'total_flos': 7.214635008e+17, 'train_loss': 1.2988064765930176, 'epoch': 25.0})

In [43]:
# Сохраняем модель
model.save_pretrained("./whisper-finetuned")
processor = WhisperProcessor(feature_extractor, tokenizer)
processor.save_pretrained("./whisper-finetuned")

[]

In [49]:
device = "cuda:0"

model_type = "finetuned"
# model_type = "pretrained"

if model_type == "finetuned":
    model = WhisperForConditionalGeneration.from_pretrained("/kaggle/working/whisper-finetuned")
    model.to(device)
    processor = WhisperProcessor.from_pretrained("/kaggle/working/whisper-finetuned")

elif model_type == "pretrained":
    # Для сравнения можем применить не дообученную модель
    model = WhisperForConditionalGeneration.from_pretrained(openai/whisper-small)
    model.to(device)
    processor = WhisperProcessor.from_pretrained(openai/whisper-small)

else:
    raise ValueError("You've passed wrong `model_type`.")

In [63]:
from jiwer import visualize_alignment, process_words


# def postprocess_text(text: str) -> str:
#     text = text.lower().strip()
#     text = "".join(s for s in text if str(s).isalpha() or s == " ")

#     return text


# def infer_whisper(wav: torch.Tensor, sample_rate: int, lang: str = "en") -> str:
#     # Обрабатываем аудио
#     inputs = processor(
#         wav.squeeze().numpy(),
#         sampling_rate=sample_rate,
#         return_tensors="pt",
#     )

#     # Генерируем транскрипцию
#     outputs = model.generate(
#         **inputs.to(device),
#         language=f"<|{lang}|>"
#     )

#     hypothesis = processor.batch_decode(outputs, skip_special_tokens=True)[0]

#     return hypothesis

def postprocess_text(text: str) -> str:
    """Постобработка строк. 
    Приводим в нижний регистр, удаляем краевые пробельные 
    и неалфавитные символы (цифры, пунктуацию).
    """

    text = text.lower().strip()
    text = "".join(s for s in text if str(s).isalpha() or s == " ")

    return text


def infer_whisper(
    model: WhisperForConditionalGeneration,
    processor: WhisperProcessor,
    wav: torch.Tensor, 
    sample_rate: int, 
    lang: str = "ru",
    device: str = "cuda:0"
) -> str:
    """Обработка Whisper-ом входного аудио."""

    # Обрабатываем аудио
    inputs = processor(
        wav.squeeze().numpy(),
        sampling_rate=sample_rate,
        return_tensors="pt",
    )
    print(inputs)
    # Генерируем транскрипцию
    outputs = model.generate(
        inputs.to(device),
        language=f"<|{lang}|>"
    )

    hypothesis = processor.batch_decode(outputs, skip_special_tokens=True)[0]

    return hypothesis

In [64]:
# Читаем аудио
wav, sr = torchaudio.load("/kaggle/input/audiodata/audio_proccesing/avito_audio_augumented/avito_aug_1000_1.wav")

display(Audio(data=wav, rate=sr))

# Запускаем модель на аудио
hypothesis = infer_whisper(wav, sr, lang="ru")
hypothesis = postprocess_text(hypothesis)

# # Обрабатываем референс
# reference = "and towards christmas he was one of the first that was cut down"
# reference = postprocess_text(reference)

# Выводим разницу текстов
# out = process_words(reference, hypothesis)
# print(visualize_alignment(out))

TypeError: infer_whisper() missing 2 required positional arguments: 'wav' and 'sample_rate'

In [65]:
# Инициализируем модель
# model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
# model.to(device)
# processor = WhisperProcessor.from_pretrained("openai/whisper-small")

# Читаем аудио
wav, sr = torchaudio.load("/kaggle/input/audiodata/audio_proccesing/audio_homework/data/audio/hello_avito.wav",)
display(Audio(data=wav, rate=sr))

# Рэсемплим в 16кГц (Whisper по-другому не умеет)
wav = torchaudio.transforms.Resample(sr, 16000)(wav)

# Запускаем модель на аудио
hypothesis = infer_whisper(
    model=model,
    processor=processor,
    wav=wav, 
    sample_rate=16000, 
    lang="ru",
    device=device
)
postprocess_text(hypothesis)

{'input_features': tensor([[[-0.5608, -0.3748, -0.4072,  ..., -1.0711, -1.0711, -1.0711],
         [-0.2437, -0.5416, -0.4970,  ..., -1.0711, -1.0711, -1.0711],
         [-0.2359, -0.5087, -0.6625,  ..., -1.0711, -1.0711, -1.0711],
         ...,
         [-1.0711, -1.0711, -1.0711,  ..., -1.0711, -1.0711, -1.0711],
         [-1.0711, -1.0711, -1.0711,  ..., -1.0711, -1.0711, -1.0711],
         [-1.0711, -1.0711, -1.0711,  ..., -1.0711, -1.0711, -1.0711]]])}


AttributeError: 